## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Current Weather,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Arlit,18.7369,7.3853,scattered clouds,97.61,26,28,10.11,NE,2021-08-01 18:58:41
1,1,Mataura,-46.1927,168.8643,broken clouds,39.47,98,64,3.74,NZ,2021-08-01 18:54:59
2,2,Kapaa,22.0752,-159.3190,light rain,80.58,83,93,1.01,US,2021-08-01 18:58:14
3,3,Ushuaia,-54.8000,-68.3000,clear sky,46.06,61,0,5.75,AR,2021-08-01 18:55:05
4,4,Skibbereen,51.5500,-9.2667,overcast clouds,62.40,66,99,11.30,IE,2021-08-01 19:04:01


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Current Weather,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,Kapaa,22.0752,-159.3190,light rain,80.58,83,93,1.01,US,2021-08-01 18:58:14
4,4,Skibbereen,51.5500,-9.2667,overcast clouds,62.40,66,99,11.30,IE,2021-08-01 19:04:01
5,5,Albany,42.6001,-73.9662,light rain,76.39,66,55,8.10,US,2021-08-01 18:55:23
6,6,Mandera,3.9366,41.8670,overcast clouds,82.85,48,95,21.90,KE,2021-08-01 19:02:17
7,7,Tadine,-21.5500,167.8833,broken clouds,66.78,56,61,12.10,NC,2021-08-01 19:04:02
8,8,Saskylakh,71.9167,114.0833,clear sky,65.25,49,0,5.37,RU,2021-08-01 19:02:53
9,9,Jemnice,49.0190,15.5699,overcast clouds,61.95,89,100,8.90,CZ,2021-08-01 19:04:02
10,10,Ghanzi,-21.5667,21.7833,clear sky,57.40,38,0,8.03,BW,2021-08-01 19:04:03
11,11,Dawei,14.0833,98.2000,moderate rain,76.91,92,100,7.72,MM,2021-08-01 19:04:03
12,12,Ochakiv,46.6128,31.5498,clear sky,77.14,67,0,10.65,UA,2021-08-01 19:04:04


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID            567
City               567
Lat                567
Lng                567
Current Weather    567
Max Temp           567
Humidity           567
Cloudiness         567
Wind Speed         567
Country            560
Date               567
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
2,Kapaa,US,80.58,light rain,22.0752,-159.3190,
4,Skibbereen,IE,62.40,overcast clouds,51.5500,-9.2667,
5,Albany,US,76.39,light rain,42.6001,-73.9662,
6,Mandera,KE,82.85,overcast clouds,3.9366,41.8670,
7,Tadine,NC,66.78,broken clouds,-21.5500,167.8833,
8,Saskylakh,RU,65.25,clear sky,71.9167,114.0833,
9,Jemnice,CZ,61.95,overcast clouds,49.0190,15.5699,
10,Ghanzi,BW,57.40,clear sky,-21.5667,21.7833,
11,Dawei,MM,76.91,moderate rain,14.0833,98.2000,
12,Ochakiv,UA,77.14,clear sky,46.6128,31.5498,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
hotel_df

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
2,Kapaa,US,80.58,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,Skibbereen,IE,62.40,overcast clouds,51.5500,-9.2667,The West Cork Hotel
5,Albany,US,76.39,light rain,42.6001,-73.9662,
6,Mandera,KE,82.85,overcast clouds,3.9366,41.8670,Kornesh Hotel and Lodging
7,Tadine,NC,66.78,broken clouds,-21.5500,167.8833,
...,...,...,...,...,...,...,...
670,Shizunai,JP,73.20,overcast clouds,42.3339,142.3669,Shizunai Eclipse Hotel
671,Millau,FR,64.38,broken clouds,44.0833,2.9167,La Villa Des Raspes
672,Cabedelo,BR,81.27,clear sky,-6.9811,-34.8339,Hotel Pigalle
674,Port Macquarie,AU,56.89,broken clouds,-31.4333,152.9167,Rydges Port Macquarie


In [10]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
2,Kapaa,US,80.58,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,Skibbereen,IE,62.40,overcast clouds,51.5500,-9.2667,The West Cork Hotel
6,Mandera,KE,82.85,overcast clouds,3.9366,41.8670,Kornesh Hotel and Lodging
9,Jemnice,CZ,61.95,overcast clouds,49.0190,15.5699,Zdeněk Kolibáč
10,Ghanzi,BW,57.40,clear sky,-21.5667,21.7833,Ghanzi Farmhouse
...,...,...,...,...,...,...,...
670,Shizunai,JP,73.20,overcast clouds,42.3339,142.3669,Shizunai Eclipse Hotel
671,Millau,FR,64.38,broken clouds,44.0833,2.9167,La Villa Des Raspes
672,Cabedelo,BR,81.27,clear sky,-6.9811,-34.8339,Hotel Pigalle
674,Port Macquarie,AU,56.89,broken clouds,-31.4333,152.9167,Rydges Port Macquarie


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))